## Import Packages

In [1]:
import glob
import warnings
import numpy as np
import xarray as xr
from datetime import datetime
warnings.filterwarnings('ignore')

## User-Defined Fields

In [2]:
AUTHOR   = 'Savannah L. Ferretti'
EMAIL    = 'savannah.ferretti@uci.edu'
FILEDIR  = '/ocean/projects/atm200007p/sferrett/Repos/monsoon-pr/data/interim'
SAVEDIR  = '/ocean/projects/atm200007p/sferrett/Repos/monsoon-pr/data/processed'
LATRANGE = (0.,30.) #(7.,22.) 
LONRANGE = (50.,90.) #(62.,78.)
SOURCES  = [
    'OBS-HR',
    # 'OBS-LR',
    # 'AWI-ESM-1-1-LR',
    # 'BCC-CSM2-MR',
    # 'CESM2',
    # 'CMCC-CM2-SR5',
    # 'CMCC-ESM2',
    # 'CanESM5',
    # 'FGOALS-g3',
    # 'GISS-E2-1-G',
    # 'IITM-ESM',
    # 'MIROC-ES2L',
    # 'MIROC6',
    # 'MPI-ESM-1-2-HAM',
    # 'MPI-ESM1-2-HR',
    # 'MPI-ESM1-2-LR',
    # 'MRI-ESM2-0',
    # 'NESM3',
    # 'NorESM2-MM',
    # 'SAM0-UNICON',
    # 'TaiESM1',
]

## Functions

In [3]:
def load(source,varname,filedir=FILEDIR,latrange=LATRANGE,lonrange=LONRANGE):
    file = f'{filedir}/{source}_{varname}.nc'
    data = xr.open_dataset(file)
    if 'lev' in data.dims:
        data = data.transpose('lev','time','lat','lon')
    data = data.sel(lat=slice(*latrange),lon=slice(*lonrange))
    return data[varname].load()

In [4]:
def calc_thetae(p,t,q):
    p0 = 1000.
    rv = 461.5
    rd = 287.04
    kappad  = 0.2854
    epsilon = rd/rv
    r  = q/(1.-q) 
    e = (t.lev*r)/(epsilon+r)
    tl = 2840./(3.5*np.log(t)-np.log(e)-4.805)+55.
    thetae = t*(p0/p)**(kappad*(1.-0.28*r))*np.exp((3.376/tl-0.00254)*1000.*r*(1.+0.81*r))
    return thetae

def calc_es(t):
    c0 = 0.6105851e+03
    c1 = 0.4440316e+02
    c2 = 0.1430341e+01
    c3 = 0.2641412e-01
    c4 = 0.2995057e-03
    c5 = 0.2031998e-05
    c6 = 0.6936113e-08
    c7 = 0.2564861e-11
    c8 = -.3704404e-13
    tc = t-273.15
    bolton = 6.112*np.exp(17.67*tc/(243.5+tc))
    poly   = (c0+tc*(c1+tc*(c2+tc*(c3+tc*(c4+tc*(c5+tc*(c6+tc*(c7+tc*c8))))))))/100.
    es = np.where(tc<-80.,bolton,poly)
    return es

def calc_qs(p,t):
    rv = 461.5
    rd = 287.04
    epsilon = rd/rv
    es = calc_es(t)
    qs = (epsilon*es)/(p-es*(1.-epsilon))
    return qs

def calc_wb(ps,pbltop,lfttop):
    pblthickness = ps-pbltop
    lftthickness = pbltop-lfttop
    wb = (pblthickness/lftthickness)*np.log((pblthickness+lftthickness)/pblthickness)
    return wb

def get_p_above(a,levels,side):
    idx    = np.searchsorted(levels,a,side=side)
    p_above = levels[np.maximum(idx-1,0)]
    return p_above

def get_p_below(a,levels,side):
    idx    = np.searchsorted(levels,a,side=side)
    p_below = levels[np.minimum(idx,len(levels)-1)]
    return p_below

def get_layer_average(data,a,b):
    a,b,data = a.load(),b.load(),data.load()
    pabove = xr.apply_ufunc(get_p_above,a,kwargs={'levels':np.array(data.lev),'side':'right'})
    pbelow = xr.apply_ufunc(get_p_below,a,kwargs={'levels':np.array(data.lev),'side':'right'})
    fabove = data.sel({'lev':pabove})
    fbelow = data.sel({'lev':pbelow})
    correction = -fabove/2*(pbelow-pabove)*(a<data.lev[-1])
    pbelow += (pbelow==pabove) 
    belowintegral = (a-pabove)*fabove+(fbelow-fabove)*(a-pabove)**2/(pbelow-pabove)/2+correction
    innerintegral = (data*(data.lev<=a)*(data.lev>=b)).integrate('lev')
    pabove = xr.apply_ufunc(get_p_above,b,kwargs={'levels':np.array(data.lev),'side':'left'})
    pbelow = xr.apply_ufunc(get_p_below,b,kwargs={'levels':np.array(data.lev),'side':'left'})
    fabove = data.sel({'lev':pabove})
    fbelow = data.sel({'lev':pbelow})
    correction = -fbelow/2*(pbelow-pabove)*(b>data.lev[0])
    pabove -= (pbelow==pabove) 
    aboveintegral = (pbelow-b)*fabove+(fbelow-fabove)*(pbelow-pabove)*(1-((b-pabove)/(pbelow-pabove))**2)/2+correction
    layeraverage  = (belowintegral+innerintegral+aboveintegral)/(a-b)
    return layeraverage

def calc_bl_terms(wb,wl,thetaeb,thetael,thetaels):
    g = 9.81
    kappal  = 3.
    thetae0 = 340.
    cape    = ((thetaeb-thetaels)/thetaels)*thetae0
    subsat  = ((thetaels-thetael)/thetaels)*thetae0
    bl      = (g/(kappal*thetae0))*((wb*cape)-(wl*subsat))
    return cape,subsat,bl

def align_pr(pr,t):
    matches = []
    for time in t.time.values:
        closestidx = np.argmin(np.abs(pr.time.values - time))
        matches.append(closestidx)
    aligned = pr.isel(time=matches)
    aligned['time'] = t.time
    return aligned

In [18]:
def dataset(pr,bl,cape,subsat,source,author=AUTHOR,email=EMAIL):
    vardata = {'pr':([*pr.dims],pr.data),
               'bl':([*bl.dims],bl.data),
               'cape':([*cape.dims],cape.data),
               'subsat':([*subsat.dims],subsat.data)}
    coords  = {'time':bl.time.data,'lat':bl.lat.data,'lon':bl.lon.data}
    data = xr.Dataset(vardata,coords)
    data.pr.attrs     = dict(long_name='Precipitation flux',units='mm/day')
    data.bl.attrs     = dict(long_name='Average buoyancy in the lower troposphere',units='m/s²')
    data.cape.attrs   = dict(long_name='Undilute plume buoyancy',units='K')
    data.subsat.attrs = dict(long_name='Subsaturation in the lower-free troposphere',units='K')
    data.time.attrs   = dict(long_name='Time')
    data.lat.attrs    = dict(long_name='Latitude',units='$^\circ$N')
    data.lon.attrs    = dict(long_name='Longitude',units='$^\circ$E')
    data.attrs        = dict(source=source,
                             description='Calculated following Ahmed & Neelin (2021) Eq. 1',
                             history=f'Created on {datetime.today().strftime("%Y-%m-%d")} by {author} ({email})')
    return data

def save(data,source,savedir=SAVEDIR):
    return data.to_netcdf(f'{savedir}/{source}_bl-pr_FULL.nc')

## Process & Save Data

In [ ]:
for source in SOURCES:
    
    pr = load(source,'pr')
    ps = load(source,'ps')
    t  = load(source,'t')
    q  = load(source,'q')
    p  = (t.lev).expand_dims({'time':t.time,'lat':t.lat,'lon':t.lon},(1,2,3))

    qs = calc_qs(p,t)
    thetae  = calc_thetae(p,t,q)
    thetaes = calc_thetae(p,t,qs)

    pbltop = ps-100.
    lfttop = xr.full_like(ps,500.)   
    wb = calc_wb(ps,pbltop,lfttop)
    wl = 1-wb
    thetaeb  = get_layer_average(thetae,ps,pbltop)*np.sqrt(-1+2*(ps>lfttop))
    thetael  = get_layer_average(thetae,pbltop,lfttop)
    thetaels = get_layer_average(thetaes,pbltop,lfttop)

    cape,subsat,bl = calc_bl_terms(wb,wl,thetaeb,thetael,thetaels)
    alignedpr = align_pr(pr,t)
    ds = dataset(alignedpr,bl,cape,subsat,source)
    save(ds,source)